In [ ]:
import cv2
import os
import numpy as np

# Složka pro uložení snapshotů
output_dir = "snapshots"
os.makedirs(output_dir, exist_ok=True)

# Stylizace
STYLE_ORIGINAL = 1
STYLE_SKETCH = 2
STYLE_CARTOON = 3
STYLE_GRAY = 4
STYLE_SEPIA = 5
STYLE_BLUR = 6
STYLE_EDGES = 7
STYLE_OIL = 8
STYLE_TOON = 9
STYLE_COOL = 0

current_style = STYLE_ORIGINAL

# Inicializace kamery
video = cv2.VideoCapture(0)
padding = 0.3

# Funkce pro efekty
def apply_sketch(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    inv_gray = 255 - gray
    blur = cv2.GaussianBlur(inv_gray, (21, 21), 0)
    sketch = cv2.divide(gray, 255 - blur, scale=256)
    return cv2.cvtColor(sketch, cv2.COLOR_GRAY2BGR)

def apply_cartoon(frame):
    small = cv2.pyrDown(frame)
    for _ in range(2):
        small = cv2.bilateralFilter(small, 9, 75, 75)
    cartoon = cv2.pyrUp(small)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    edges = cv2.adaptiveThreshold(cv2.medianBlur(gray, 7), 255,
                                  cv2.ADAPTIVE_THRESH_MEAN_C,
                                  cv2.THRESH_BINARY, 9, 10)
    edges_colored = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
    return cv2.bitwise_and(cartoon, edges_colored)

def apply_gray(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)

def apply_sepia(frame):
    kernel = np.array([[0.272, 0.534, 0.131],
                       [0.349, 0.686, 0.168],
                       [0.393, 0.769, 0.189]])
    sepia = cv2.transform(frame, kernel)
    return np.clip(sepia, 0, 255).astype(np.uint8)

def apply_blur(frame):
    return cv2.GaussianBlur(frame, (15, 15), 0)

def apply_edges(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    return cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

def apply_oil(frame):
    return cv2.edgePreservingFilter(frame, flags=1, sigma_s=60, sigma_r=0.4)

def apply_toon(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hsv[..., 1] = cv2.add(hsv[..., 1], 40)
    bright = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    cartoon = apply_cartoon(bright)
    return cartoon

def apply_cool(frame):
    b, g, r = cv2.split(frame)
    b = cv2.add(b, 30)
    r = cv2.subtract(r, 30)
    cool = cv2.merge((b, g, r))
    return np.clip(cool, 0, 255).astype(np.uint8)

print("\n🔁 Klávesy pro přepínání stylů:")
print("  1 = Originál")
print("  2 = Tužková skica")
print("  3 = Cartoon")
print("  4 = Černobílý")
print("  5 = Sepia")
print("  6 = Rozmazání")
print("  7 = Hrany (Canny)")
print("  8 = Oil Painting")
print("  9 = Komiks (Toon)")
print("  0 = Cool Tones")
print("  s = Uložit snímek")
print("  q = Ukončit")

snapshot_counter = 1

while True:
    ret, frame = video.read()
    frame = cv2.flip(frame, 1)
    output = frame.copy()

    if current_style == STYLE_SKETCH:
        output = apply_sketch(output)
    elif current_style == STYLE_CARTOON:
        output = apply_cartoon(output)
    elif current_style == STYLE_GRAY:
        output = apply_gray(output)
    elif current_style == STYLE_SEPIA:
        output = apply_sepia(output)
    elif current_style == STYLE_BLUR:
        output = apply_blur(output)
    elif current_style == STYLE_EDGES:
        output = apply_edges(output)
    elif current_style == STYLE_OIL:
        output = apply_oil(output)
    elif current_style == STYLE_TOON:
        output = apply_toon(output)
    elif current_style == STYLE_COOL:
        output = apply_cool(output)

    style_texts = {
        STYLE_ORIGINAL: "Originál",
        STYLE_SKETCH: "Tužková skica",
        STYLE_CARTOON: "Cartoon",
        STYLE_GRAY: "Černobílý",
        STYLE_SEPIA: "Sepia",
        STYLE_BLUR: "Rozmazání",
        STYLE_EDGES: "Hrany",
        STYLE_OIL: "Oil Painting",
        STYLE_TOON: "Komiks",
        STYLE_COOL: "Cool Tones"
    }

    cv2.putText(output, f"Styl: {style_texts.get(current_style, 'Neznámý')}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

    cv2.imshow("Webcam stylizace", output)

    key = cv2.waitKey(1) & 0xFF

    if key == ord('q'):
        break
    elif key in [ord(str(i)) for i in range(10)]:
        current_style = int(chr(key))
    elif key == ord('s'):
        filename = os.path.join(output_dir, f"snapshot_{snapshot_counter}.jpg")
        cv2.imwrite(filename, output)
        print(f"🖼️ Snapshot uložen jako {filename}")
        snapshot_counter += 1

video.release()
cv2.destroyAllWindows()

2025-03-22 18:22:30.817 python[14111:285402] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.



🔁 Klávesy pro přepínání stylů:
  1 = Originál
  2 = Tužková skica
  3 = Cartoon
  4 = Černobílý
  5 = Sepia
  6 = Rozmazání
  7 = Hrany (Canny)
  8 = Oil Painting
  9 = Komiks (Toon)
  0 = Cool Tones
  s = Uložit snímek
  q = Ukončit


2025-03-22 18:22:31.807 python[14111:285402] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-22 18:22:31.807 python[14111:285402] +[IMKInputSession subclass]: chose IMKInputSession_Modern


🖼️ Snapshot uložen jako snapshots/snapshot_1.jpg
🖼️ Snapshot uložen jako snapshots/snapshot_2.jpg
🖼️ Snapshot uložen jako snapshots/snapshot_3.jpg
🖼️ Snapshot uložen jako snapshots/snapshot_4.jpg
🖼️ Snapshot uložen jako snapshots/snapshot_5.jpg
🖼️ Snapshot uložen jako snapshots/snapshot_6.jpg
🖼️ Snapshot uložen jako snapshots/snapshot_7.jpg
🖼️ Snapshot uložen jako snapshots/snapshot_8.jpg
🖼️ Snapshot uložen jako snapshots/snapshot_9.jpg
🖼️ Snapshot uložen jako snapshots/snapshot_10.jpg


: 